# Test Notebook

notebook for testing the toctoc api and prompts for gpt

In [64]:
%load_ext dotenv
%dotenv
%run db.ipynb
%run download_json.ipynb

The dotenv extension is already loaded. To reload it, use:
  %reload_ext dotenv
{"status":"ok","statusCode":200,"message":"","docs":[{"_id":"6774f7978b2ddfa7e911a581","address":{"lat":-33.4187,"long":-70.5855,"communeName":"Las Condes","street":"Townhouse proyecto Net Zero en Málaga, El Golf  "},"area":{"balcony":0,"constructed":0,"land":231.4,"usable":163.4},"distance":353.2404633197919,"operationFamily":"Venta Usado","propertyFamilyType":"Casas","price":24490,"salableArea":163.4,"ufm2":149.87760097919215,"publicationDate":"2024-12-16T00:00:00.000Z"},{"_id":"6774f7978b2ddfa7e911a57f","address":{"lat":-33.4189,"long":-70.5867,"communeName":"Las Condes","street":"Moderno y vanguardista Townhouse en El Golf  "},"area":{"balcony":0,"constructed":0,"land":233.1,"usable":174.38},"distance":280.5717238235065,"operationFamily":"Venta Usado","propertyFamilyType":"Casas","price":24800,"salableArea":174.38,"ufm2":142.21814428260123,"publicationDate":"2024-12-16T00:00:00.000Z"},{"_id":"6774f7e38b

In [65]:
etapa_1_json = {
    "search": "Quiero buscar una propiedad",   
} 
etapa_2_json = {
    "search": "Quiero buscar una propiedad",
    "customResponse": "",
    "intent": "Buscar propiedad",
    "workflow": "Buscar",
    "response": [
        {"key": "commune", "value": ""},
        {"key": "region", "value": ""},
        {"key": "typeOfProperty", "value": ""},
        {"key": "typeOfOperation", "value": ""},
        {"key": "bedrooms", "value": ""},
        {"key": "bathrooms", "value": ""},
        {"key": "parking", "value": ""},
        {"key": "storage", "value": ""},
        {"key": "area", "value": ""},
        {"key": "priceMax", "value": ""},
        {"key": "priceMin", "value": ""},
        {"key": "petFriendly", "value": ""}
    ],
    "questions": [{"key": "", "value": "bool"}]
} 

etapa_3_json = {
    "search": "Quiero buscar una propiedad",
    "customResponse": "",
    "intent": "Buscar propiedad",
    "workflow": "Buscar",
    "response": [
        {"key": "commune", "value": ""},
        {"key": "region", "value": ""},
        {"key": "typeOfProperty", "value": ""},
        {"key": "typeOfOperation", "value": ""},
        {"key": "bedrooms", "value": ""},
        {"key": "bathrooms", "value": ""},
        {"key": "parking", "value": ""},
        {"key": "storage", "value": ""},
        {"key": "area", "value": ""},
        {"key": "priceMax", "value": ""},
        {"key": "priceMin", "value": ""},
        {"key": "petFriendly", "value": ""}
    ],
    "questions": [
        {"key": "commune", "value": True},
        {"key": "region", "value": True},
        {"key": "typeOfProperty", "value": True},
        {"key": "typeOfOperation", "value": True},
        {"key": "bedrooms", "value": True},
        {"key": "bathrooms", "value": True},
        {"key": "parking", "value": True},
        {"key": "storage", "value": True},
        {"key": "area", "value": True},
        {"key": "priceMax", "value": True},
        {"key": "priceMin", "value": True},
        {"key": "petFriendly", "value": True}
    ]
} 

In [66]:
sys_prompt = """
You are a helpful customer support assistant for the TOCTOC site. Your role is to assist customers effectively by using the provided tools and determining their intent.

### Objectives:
1. **Determine Customer Intent**: Identify whether the customer's goal is:
   - 'Buscar propiedad' (Search for a property)
   - 'Tasar propiedad' (Value a property)
   - 'Financiar' (Finance a property)

2. **Provide Assistance Based on Intent**:
   - If the intent is **'Buscar propiedad'**:
     - Narrow down the search as much as possible based on customer-provided information.
     - Use the `get_properties` function to retrieve properties based on the provided attributes, such as region, commune, and other filters. Ensure the search is specific and relevant to the user's needs.
   - If the intent is **'Tasar propiedad'** or **'Financiar'**:
     - Use the relevant tools and functions to assist with the task, ensuring the required arguments are provided.

3. **Handle Region and Commune Parameters**:
   - If the user provides a **commune** or **city**, and you can derive the corresponding region:
     - Immediately call the `get_properties` function with the parameters.

4. **Generate JSON Responses**:
   - Always use the `make_json` function to format your responses.
   - If the conversation is complete, save the JSON file using the `download_json` function.

### Rules:
- Always confirm the user's intent explicitly before proceeding with any actions.
- Ask follow-up questions to refine search parameters or gather necessary details for tasks.
- Avoid unnecessary actions or calls if insufficient information is provided.

### Example Interactions:
#### Example 1: Searching for a Property
**Customer**: "I'm looking for a property to rent in Santiago."
**Assistant**: Identifies intent as 'Buscar propiedad', extracts 'Santiago' as the commune, determines the region, and calls `get_properties`.

#### Example 2: Valuing a Property
**Customer**: "I want to know how much my house in Viña del Mar is worth."
**Assistant**: Identifies intent as 'Tasar propiedad' and uses the provided tools to assist.

#### Example 3: Financing
**Customer**: "Can you help me finance a property?"
**Assistant**: Identifies intent as 'Financiar' and collects necessary details to proceed with the task.

---

Focus on providing clear, concise, and helpful responses while adhering to these guidelines.

      """

In [58]:
tools = [
  {
      "type": "function",
      "function": {

          "name": "get_properties",
          "parameters": {
              "type": "object",
              "properties": {
                  "city": {"type": "string",
                                           "description": "commune specified by customer."},
                  "region":{"type": "string",
                            "enum":[
                                 "Región de Arica y Parinacota",
                                "Región de Tarapacá",
                                "Región de Antofagasta",
                                "Región de Atacama",
                                "Región de Coquimbo",
                                "Región de Valparaíso",
                                "Región Metropolitana de Santiago",
                                "Región del Libertador Gral. Bernardo O'Higgins",
                                "Región del Maule",
                                "Región de Ñuble",
                                "Región del Biobío",
                                "Región de La Araucanía",
                                "Región de Los Ríos",
                                "Región de Los Lagos",
                                "Región Aysén del G. Carlos Ibáñez del Campo",
                                "Región de Magallanes y de la Antártica Chilena"
                            ],
                                      "description": "Geographical Region, if not specified provide the region that you think the customer is refferring to. Remeber, regions of Chile"},
    
              },
              "additionalProperties": False,
          },
      },
  },
  {
      "type": "function",
      "function": {

          "name": "make_json",
          "parameters": {
              "type": "object",
              "properties": {
                  "search": {"type": "string",
                                           "description": "Is the lattest message submited by the user"},
                  "customResponse": {
                      "type": "string",
                      "description": "Latest message sent by the bot."
                  },
                  "intent":{
                      "type": "string",
                      "description": "The intent of the customer in this conversation, it can be 'Buscar propiedad', 'Financiar' or 'Tasar propiedad"
                  },
                  "workflow": {
                      "type":"string",
                      "description": "Simpler version of intent, for the intent 'Buscar propiedad' is 'Buscar', for 'Financiar' is 'Hipotecario and for 'Tasar propiedad' is 'Tasar'."
                  },
                  "response":{
                      "type": "array",
                      "items":{ "type":  "object",
                              "properties":{
                                  "key":{"type":"string",
                                         "enum": [
                                            "commune",
                                            "region",
                                            "typeOfProperty",
                                            "typeOfOperation",
                                            "bedrooms",
                                            "bathrooms",
                                            "parking",
                                            "storage",
                                            "area",
                                            "priceMax",
                                            "priceMin",
                                            "petFriendly",
                                            ]},
                                  "value":{"type":"string"}}},
                      "description": "Each element has a dictionary with a 'key' as an attribute and a 'value' parameter, that is the value for the key. For example: \
                          {'key': 'commune', 'value': 'Huechuraba'}"
                      },
                  "questions":{
                      "type": "array",
                      "items":{ "type":  "object",
                              "properties":{
                                  "key":{"type":"string",
                                         "enum": [
                                            "commune",
                                            "region",
                                            "typeOfProperty",
                                            "typeOfOperation",
                                            "bedrooms",
                                            "bathrooms",
                                            "parking",
                                            "storage",
                                            "area",
                                            "priceMax",
                                            "priceMin",
                                            "petFriendly",
                                            ]},
                                  "value":{"type":"boolean"}}},
                      "description": "Each element has a dictionary with a 'key' as an attribute and a 'value' parameter is true if the response has a value o either \
                          is false if it has an empty string. For example: \
                          {'key': 'commune', 'value': true} \
                          or if in response does not have a value for commune,h it would be:\
                          {'key': 'commune', 'value': false}"
                  }
              },
              "additionalProperties": False,
          },
      },
  },
     {"type": "function",
      "function": {

          "name": "download_json",
          "parameters": {
              },
              "additionalProperties": False,
          },
      },
  
]

In [ ]:
from openai import OpenAI
from dotenv import load_dotenv
import os
import json
from json_gen import make_json
load_dotenv()

api_key_openai = os.getenv('API_KEY_OPENAI')
client = OpenAI(api_key=api_key_openai)


message = input("send a message for our bot >:)")
completion = client.chat.completions.create(
  model="gpt-4o",
  messages=[{
      "role": "system",
      "content": sys_prompt,
  },
      {"role": "user", "content": message}],

  tools=tools,
)
calls = completion.choices[0].message.tool_calls
arguments = json.loads(calls[0].function.arguments)


In [63]:
print(calls)

[ChatCompletionMessageToolCall(id='call_lXxL8c8MpItrw4I8nyxXkTO1', function=Function(arguments='{"city":"La Serena","region":"Región de Coquimbo"}', name='get_properties'), type='function')]


In [61]:
print(arguments)

{'city': 'La Serena', 'region': 'Región de Coquimbo'}


In [62]:
print(completion.choices)

[Choice(finish_reason='tool_calls', index=0, logprobs=None, message=ChatCompletionMessage(content='Para confirmar, estás interesado en "Buscar propiedad" para arrendar un departamento en La Serena. Permíteme buscar las opciones disponibles para ti con los siguientes criterios:\n\n- Ubicación: La Serena, Región de Coquimbo\n- Tipo de propiedad: Departamento\n- Operación: Arriendo\n- Dormitorios: 3\n- Baños: 2\n- Precio máximo: 450,000 CLP\n- Superficie: 50 a 100 m²\n\nVoy a buscar esta información y volveré enseguida.', refusal=None, role='assistant', audio=None, function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_lXxL8c8MpItrw4I8nyxXkTO1', function=Function(arguments='{"city":"La Serena","region":"Región de Coquimbo"}', name='get_properties'), type='function')]))]
